In [3]:
# Import packages needed for Predict Challenge
import pandas as pd
import numpy as np
import nltk
import string
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from sklearn.metrics import f1_score
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [1]:
from google.colab import files
uploaded = files.upload()

Saving train.csv to train.csv


In [4]:
train = pd.read_csv('train.csv')
train.head()

,sentiment,message,tweetid
0,1,PolySciMajor EPA chief doesn't think carbon di...,625221
1,1,It's not like we lack evidence of anthropogeni...,126103
2,2,RT @RawStory: Researchers say we have three ye...,698562
3,1,#TodayinMaker# WIRED : 2016 was a pivotal year...,573736
4,1,"RT @SoyNovioDeTodas: It's 2016, and a racist, ...",466954


In [5]:
from sklearn.utils import resample

In [6]:
pos_sentiment = train[train['sentiment']==1]
neg_sentiment = train[train['sentiment']==-1]
neutral = train[train['sentiment']==0]
news = train[train['sentiment']==2]

# Upsample minority
neg_sentiment_upsampled = resample(neg_sentiment, replace=True, n_samples=len(pos_sentiment), random_state=27)
neutral_upsampled = resample(neutral, replace=True, n_samples=len(pos_sentiment), random_state=27)
news_upsampled = resample(news, replace=True, n_samples=len(pos_sentiment), random_state=27)

In [7]:
# Combine Majority and Upsampled Minority Datasets
upsampled = pd.concat([pos_sentiment, neg_sentiment_upsampled, neutral_upsampled, news_upsampled])

In [8]:
y = upsampled['sentiment']
X = upsampled['message']

In [9]:
vectorizer = TfidfVectorizer(ngram_range=(1,2), min_df=2, stop_words='english')
X_vectorized = vectorizer.fit_transform(X)

In [10]:
X_train,X_val,y_train,y_val = train_test_split(X_vectorized,y,test_size=0.3,shuffle=True, random_state=27)

In [11]:
from sklearn.svm import SVC

In [12]:
sv = SVC(gamma=0.1, C=1)

In [13]:
sv.fit(X_train, y_train)
sv_pred = sv.predict(X_val)

In [14]:
f1_score(y_val, sv_pred, average='macro')

0.8814557785988908

In [15]:
from google.colab import files
uploaded = files.upload()

Saving test.csv to test.csv


In [16]:
test = pd.read_csv('test.csv')
test.head()

,message,tweetid
0,Europe will now be looking to China to make su...,169760
1,Combine this with the polling of staffers re c...,35326
2,"The scary, unimpeachable evidence that climate...",224985
3,@Karoli @morgfair @OsborneInk @dailykos \nPuti...,476263
4,RT @FakeWillMoore: 'Female orgasms cause globa...,872928


In [17]:
testx = test['message']
test_vect = vectorizer.transform(testx)

In [18]:
y_pred = sv.predict(test_vect)

In [19]:
test['sentiment'] = y_pred

In [20]:
test.head()

,message,tweetid,sentiment
0,Europe will now be looking to China to make su...,169760,2
1,Combine this with the polling of staffers re c...,35326,2
2,"The scary, unimpeachable evidence that climate...",224985,2
3,@Karoli @morgfair @OsborneInk @dailykos \nPuti...,476263,1
4,RT @FakeWillMoore: 'Female orgasms cause globa...,872928,0


In [21]:
testsub = pd.DataFrame(y_pred, columns=['sentiment'])
testsub.head()

,sentiment
0,2
1,2
2,2
3,1
4,0


In [22]:
from google.colab import files
uploaded = files.upload()

Saving tweetid.csv to tweetid.csv


In [23]:
tweet = pd.read_csv('tweetid.csv')
tweet.head()

,tweetid
0,169760
1,35326
2,224985
3,476263
4,872928


In [24]:
output = pd.DataFrame({"tweetid":tweet['tweetid']})
testsubmissionsvc3 = output.join(testsub)
testsubmissionsvc3.to_csv("testsubmissionsvc3.csv", index=False)

In [26]:
testsubmissionsvc3.head()

,tweetid,sentiment
0,169760,2
1,35326,2
2,224985,2
3,476263,1
4,872928,0
